<a href="https://colab.research.google.com/github/krMaynard/genAI/blob/main/Chrome_Extension_On_Device_AI_Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Please ensure you have imported a Gemini API key from AI Studio.
You can do this directly in the Secrets tab on the left.

After doing so, please run the setup cell below.

In [1]:
!pip install -U -q "google"
!pip install -U -q "google.genai"

import os
from google.colab import userdata
from google.colab import drive
os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")

drive.mount("/content/drive")
# Please ensure that uploaded files are available in the AI Studio folder or change the working folder.
os.chdir("/content/drive/MyDrive/Google AI Studio")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 8.9 MB/s eta 0:00:00
Mounted at /content/drive


# Code Generation

In [3]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import os
from google import genai
from google.genai import types

prompt = """Code a Chrome extension that uses the built in Chrome LLM API to answer questions about the page and runs in a side panel, using the Chrome Side Panel API."""


def generate():
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.5-pro"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=prompt),
            ],
        ),
    ]
    tools = [
        types.Tool(url_context=types.UrlContext()),
        types.Tool(googleSearch=types.GoogleSearch(
        )),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=0,
        thinking_config = types.ThinkingConfig(
            thinking_budget=-1,
        ),
        tools=tools,
        response_mime_type="text/plain",
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end="")

if __name__ == "__main__":
    generate()


## Chrome Extension: Page Q&A with Built-in LLM

This Chrome extension empowers you to ask questions about the content of the current web page and receive answers directly within a convenient side panel. Leveraging Chrome's built-in Large Language Model (LLM), this tool provides a seamless and efficient way to interact with and understand web content without leaving your browser tab.

### Code Breakdown

To create this extension, you will need the following four files:

**1. `manifest.json`**

This file serves as the blueprint for the extension, defining its name, version, permissions, and the user interface components.

```json
{
  "name": "Page Q&A with Chrome LLM",
  "version": "1.0",
  "manifest_version": 3,
  "description": "Answers questions about the current page using Chrome's built-in LLM.",
  "permissions": [
    "sidePanel",
    "activeTab",
    "scripting",
    "ai"
  ],
  "side_panel": {
    "default_path": "sidepanel.html"
  },
  "action": {
    "default_title": "Ask a qu